# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок


## Шаг 1. Откройте файл с данными и изучите общую информацию

In [174]:
import pandas as pd

In [175]:
#import pandas as pd
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение их датафрейме 
display(data.head(10)) # вывод первых 10 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [176]:
data.info() # вывод общей информации о таблице данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
В таблице 12 столбцов:
* `children` — количество детей в семье;
* `days_employed` — трудовой стаж в днях;  
* `dob_years` — возраст клиента;
* `education` — образование клиента;
* `education_id` — идентификатор образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолжность по возврату кредитов;
* `total_income` — доход в месяц;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается-значит есть пропуски, которые необходимо обработать.
Также данные по трудовому стажу со знаком минус, и указаны в днях, что довольно сложно воспринимать(в некоторых столбцах странные значения н-р: 340266.072047/365 = 932 года, считаю что есть ошибка заполнения данных).
Данные об образовании заполнены с разным регистром.
Данные по трудовому стажу и доходу указаны в дробных числах, для восприятия информации необходимо перевести их в целочисленный тип int.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [177]:
display(data.isna().sum()) # вывод количества пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Заполню пропуски в столбце "доход" с помощью loc с группировкой по одной переменной.**

In [178]:
for income_type in data['income_type'].unique(): #с помощью цикла посчитаем медианый доход для каждого типа занятости
    median_income = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, median_income)# вывод медианного дохода для каждой группы занятости
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income_type), 'total_income'] = median_income #выбираем ячейки с пропусками и типом занятости в цикле и заменяем соответсвующую ячейку с доходом на медианный доход
print(data['total_income'].isna().sum()) # проверяем количество пропусков в столбце "доход"            

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
0


In [179]:
data['days_employed'] = data['days_employed'].abs() # вывод значений столбца трудовой стаж по модулю
data['days_employed'] = data['days_employed'] / 365 # переведем трудовой стаж с дней в годы
data = data.rename(columns = {'days_employed': 'years_employed'}) # заменим название столбца трудовой стаж в днях на трудовой стаж в годах
print(data['years_employed'].median()) # медианный трудовой стаж составил 6.01 лет


data['years_employed'] = data['years_employed'].fillna(data['years_employed'].median()) # заменяем пропуски на медианный трудовой стаж

display(data.isna().sum()) # проверяем что пропусков больше не осталось

6.0115631969279315


children            0
years_employed      0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**добавила пересчет медианного дохода для разных типов занятости

**Вывод**

Пропуски были только в столбцах трудовой стаж и доход в месяц. Считаю, что для получения одобрения по кредиту эти данные обязательно должны были заполняться сотрудником банка, предположу что пропуски появились из-за утери данных. Приму способ их заполнения как медианный, потому как в данных по трудовому стажу есть выбросы по значениям, соответственно метод mean() не даст объективной картины, и также для оценки среднего дохода этот метод отражает более реальные показатели. 

### Замена типа данных

In [180]:
# для замены типа данных будем использовать метод astype()
data['years_employed'] = data['years_employed'].astype('int') # заменяем тип данных столбца трудовой стаж с float на int
data['total_income'] = data['total_income'].astype('int') # заменяем тип данных столбца доход с float на int


In [181]:
data.info() # проверяем что тип данных в этих двух столбцах изменился


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
years_employed      21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

После замены типа данных на целочисленный таблица приобретает более наглядный вид.

### Обработка дубликатов

In [182]:
display(data.duplicated().sum()) # подсчёт явных дубликатов


54

Дубликатов 54, на всю выборку всего 0,25 %. Предоложим, что они появились из-за повторного внесения данных, но чисто теоретически можно предположить, что среди клиентов банка могли быть клиентами с одинаковыми анкетными данныеми. Считаю, что дубликаты можно удалить не просматривая, удаление не повлияет на результаты исследования из-за маленького количества. 

In [183]:
data = data.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)


In [184]:
display(data.duplicated().sum()) # проверка на отсутствие дубликатов


0

Теперь поищем неявные дубликаты. С помощью метода value_counts() в столбце "семейное положение" и столбце "цель кредита" (выбираю только эти столбцы, потому что их данные нужны для ответа на вопросы исследования) проанализируем значения.

In [185]:
display(data['family_status'].unique()) # вывод уникальныx значений столбца "цель кредита"

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [186]:
display(data['purpose'].value_counts().to_frame()) # вывод уникальныx значений столбца "семейное положение"

,purpose
свадьба,793
на проведение свадьбы,773
сыграть свадьбу,769
операции с недвижимостью,675
покупка коммерческой недвижимости,662
покупка жилья для сдачи,652
операции с жильем,652
операции с коммерческой недвижимостью,650
жилье,646
покупка жилья,646


Повторяющихся значений  в столбце "семейное положение" нет. Можно привести значения в столбце "семейное положение" к нижнему регистру. Повторяющихся значений в столбце "цель кредита" также нет, но есть похожие по смыслу. 

In [187]:
data['family_status'] = data['family_status'].str.lower() # приведем к нижнему регистру значения в столбце "семейное положение"
data['education'] = data['education'].str.lower() # приведем к нижнему регистру значения в столбце "образование"

In [188]:
display(data.head(10))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [189]:
data = data.drop_duplicates().reset_index(drop = True)

In [190]:
display(data.duplicated().sum())

0

**Вывод**

Избавившись от дубликатов можно анализировать значения столбцов "cемейное положение" и "цель кредита", чтобы ответить на вопросы исследования. В столбце "цель кредита" достаточно похожих по смыслу значений, соответственно для дальнейшего исследования необходимо их разбить на категории.

### Лемматизация

Чтобы ответить на вопрос как разные цели влияют на возврат кредита, необходимо разделить цели по категориям. Разделять на категории будем с помощью выделения лемм (основ слов). Функция лемматизация находит через подключенный словарь основу переданного ей слова.

In [191]:
from pymystem3 import Mystem # импорт библиотеки лемм на русском языке
m = Mystem()

lemmas = data['purpose'].apply(m.lemmatize) # применим функцию лемматизация для всех строчек столбца "цель кредита"

print(lemmas) # получился список списков основ слов   



0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21454, dtype: object


In [192]:
lemmas_list = [] # перенесем все значения из списка списков lemma в список lemmas_list
for row in lemmas: # с помощью цикла
    for elem in row:
        lemmas_list.append(elem)
     
from collections import Counter # импорт коллекции Counter, которая формирует словарь с леммами
print(Counter(lemmas_list)) #подсчет значений в списке lemmas_list 
  

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Проанализируем все полученные леммы и выделим основные группы целей кредитов:
- на приобретение недвижимости. Список лемм: жилье, недвижимость, строительство, жилой
- на приобретение автомобиля. Список лемм: автомобиль
- на получение образования. Список лемм: образование
- на свадьбу. Список лемм: свадьба

Напишем функицию, которая будет определять категорию столбца "цель кредита" в зависимости от полученной леммы

In [193]:


def lemmas_group(row):
         
        lemma = m.lemmatize(row)    
        if 'жилье' in lemma:
            return 'на приобретение недвижимости'
        if 'недвижимость' in lemma:
            return 'на приобретение недвижимости'
        if 'строительство' in lemma:
            return 'на приобретение недвижимости'
        if 'жилой' in lemma:
            return 'на приобретение недвижимости'
        if 'автомобиль' in lemma:
            return 'на приобретение автомобиля'
        if 'образование' in lemma:
            return 'на получение образования'
        if 'свадьба' in lemma:
            return 'на свадьбу'

   
          

In [194]:
data['group_purpose'] = data['purpose'].apply(lemmas_group) # применим функцию к столбцу "цель кредита"       
display(data.head()) # выведем пять первых строчек таблицы

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,group_purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,на приобретение недвижимости
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,на приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,на приобретение недвижимости
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,на получение образования
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,на свадьбу


В существующую таблицу добавился еще один столбец "группа цели кредита"

**Вывод**

Основной задачей этого раздела было разделение на категории целей кредита с помощью лемматизации. Выделены 4 категории целей кредитов: на приобретение недвижимости, на приобретение автомобиля, на получение образования, на свадьбу.

### Категоризация данных

Для того чтобы ответить на первый вопрос исследования и третий необходимо всех поделить на группы "без детей" и "с детьми", "доход ниже среднего" и "доход выше среднего"

In [195]:
def children_id(row): #функция для категоризации данных по статусу наличия и отсутствия детей
    if row == 0:
        return 'без детей'
    return 'с детьми'
    

data['children_group'] = data['children'].apply(children_id) # добавление столбца с категоризацией по детям в таблицу


In [196]:
display(data.groupby('children_group')['debt'].count().to_frame()) # расчет количества людей с детьми и без детей

,debt
children_group,
без детей,14091
с детьми,7363


In [197]:
print(data['total_income'].median()) #вывод медианы дохода заемщиков

142594.0


In [198]:
def income_id(row): # функция для категоризации данных по доходу
    if row > 142594: # за средний доход возьмем посчитанный доход по медиане
        return 'доход выше среднего'
    return 'доход ниже среднего'
    

data['income_group'] = data['total_income'].apply(income_id) # добавление столбца с категоризацией по доходу в таблицу

In [199]:
display(data.groupby('income_group')['debt'].count().to_frame()) # расчет количества людей по разным категориям дохода

,debt
income_group,
доход выше среднего,10611
доход ниже среднего,10843


In [200]:
data_pivot_children = data.pivot_table(index = ['children_group'], columns = 'debt', values = 'children', aggfunc = 'count')  
# получение сводной таблицы по срокам возврата кредита по клиентам с детьми и без детей 

In [201]:
display(data_pivot_children) # вывод сводной таблицы по срокам возврата кредита по клиентам с детьми и без детей 

debt,0,1
children_group,,
без детей,13028,1063
с детьми,6685,678


Для ответа на второй вопрос исследования есть ли зависимость между семейным положением и возвратом кредита сделаем сводную таблицу по семейным статусам и статусу по сроку возврата кредита

In [202]:
data_pivot_family_status = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'children', aggfunc = 'count')  
# получение сводной таблицы по срокам возврата кредита по клиентам с разным семейным положением

In [203]:
display(data_pivot_family_status) # вывод сводной таблицы по срокам возврата кредита по клиентам с разным семейным положением 

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


Для ответа на третий вопрос исследования есть ли зависимость между уровнем дохода и возвратом кредита сделаем сводную таблицу по категорям дохода и статусу по сроку возврата кредита

In [204]:
data_pivot_incom_status = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'children', aggfunc = 'count')  
# получение сводной таблицы по срокам возврата кредита по клиентам с разным уровнем дохода

In [205]:
display(data_pivot_incom_status) # вывод сводной таблицы по срокам возврата кредита по клиентам с разным семейным положением 

debt,0,1
income_group,,
доход выше среднего,9780,831
доход ниже среднего,9933,910


Для подготовки ответа на четверый вопрос исследования есть ли зависисмость между целью кредита и сроком возврата, также выведем сводную таблицу по данным параметрам

In [206]:
data_pivot_group_purpose = data.pivot_table(index = ['group_purpose'], columns = 'debt', values = 'children', aggfunc = 'count')  
# получение сводной таблицы по срокам возврата кредита по клиентам с разными целями кредита

In [207]:
display(data_pivot_group_purpose) # вывод сводной таблицы по срокам возврата кредита по клиентам с разными целями кредита 

debt,0,1
group_purpose,,
на получение образования,3643,370
на приобретение автомобиля,3903,403
на приобретение недвижимости,10029,782
на свадьбу,2138,186


**Вывод**

Для ответа на вопросы исследования подготовлены сводные таблицы по количеству людей по разным группам категорий в зависимости от статуса отсутствия или наличия задолжности по кредиту. 

**Вывод по шагу предобработки данных**
Целью предобработки данных была подготовка данных к исследованию. Были устанены дубликаты, пропуски были заменены на медианы в столбцах данных необходимых для исследования, были изменены типы данных, и их вид для лучшего восприятия и обработки, данные разбиты на категории.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [208]:
#добавим столбцы с расчетом процентов в сводную таблицу
data_pivot_children['debt_0_children'] = data_pivot_children[0] * 100 / (data_pivot_children[0] + data_pivot_children[1]) #столбец с процентом возврата в срок

data_pivot_children['debt_1_children'] = data_pivot_children[1] * 100 / (data_pivot_children[0] + data_pivot_children[1])  #столбец с процентом по долгу

In [209]:
display(data_pivot_children.sort_values(by = 'debt_1_children')) # вывод сводной таблицы по срокам 
                                                                 # возврата кредита по клиентам с детьми и без детей



debt,0,1,debt_0_children,debt_1_children
children_group,,,,
без детей,13028,1063,92.456178,7.543822
с детьми,6685,678,90.791797,9.208203


**Вывод**

Исходя из сводной таблицы (столбец 'debt_1_children') видим, что процент должников больше в категории людей с детьми и составляет 9.2 %. Нужно отметить, что разница между категориями составляет 1,67 % и можно сделать вывод, с наличием детей у заемщика появляется риск задолжности по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [210]:
#добавим столбцы с расчетом процентов в сводную таблицу по категориями людей по семейным статусам
data_pivot_family_status['debt_0_family_status'] = data_pivot_family_status[0] * 100 / (data_pivot_family_status[0] + data_pivot_family_status[1])

data_pivot_family_status['debt_1_family_status'] = data_pivot_family_status[1] * 100 / (data_pivot_family_status[0] + data_pivot_family_status[1])


In [211]:
display(data_pivot_family_status.sort_values(by = 'debt_1_family_status')) # вывод сводной таблицы по срокам возврата кредита
                                                                           # по клиентам с разными семейными статусами



debt,0,1,debt_0_family_status,debt_1_family_status
family_status,,,,
вдовец / вдова,896,63,93.430657,6.569343
в разводе,1110,85,92.887029,7.112971
женат / замужем,11408,931,92.454818,7.545182
гражданский брак,3763,388,90.652855,9.347145
не женат / не замужем,2536,274,90.249110,9.750890


**Вывод**

Из таблицы видно, что больший процент задолжности среди не женатых/ не замужних -9.75%. К не женатым также можно отнести группы людей "в разводе", "вдовец/вдова", "гражданский брак". Тогда посчитав средний процент по задолжности кредита на эти группы, получим 8,18 %, что на 0.65 % больше, чем процент задолжности у женатых/ замужних. Из-за несущественной разницы процнтов, можно сделать вывод, что статус семейного положения не влияет на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [212]:
# добавим столбцы с расчетом процентов в сводную таблицу
data_pivot_incom_status['debt_0_incom_status'] = data_pivot_incom_status[0] * 100 / (data_pivot_incom_status[0] + data_pivot_incom_status[1]) #столбец с процентом возврата в срок
data_pivot_incom_status['debt_1_incom_status'] = data_pivot_incom_status[1] * 100 / (data_pivot_incom_status[0] + data_pivot_incom_status[1])  #столбец с процентом по долгу

In [213]:
display(data_pivot_incom_status.sort_values(by = 'debt_1_incom_status')) # вывод сводной таблицы по срокам возврата кредита 
                                                                         # по клиентам с разными уровнями дохода




debt,0,1,debt_0_incom_status,debt_1_incom_status
income_group,,,,
доход выше среднего,9780,831,92.168504,7.831496
доход ниже среднего,9933,910,91.607489,8.392511


**Вывод**

По сводной таблице можно отметить, что процент не вернувших вовремя кредит, больше среди тех, у кого доход ниже среднего на 0,33%. Стоить отметить, что разница по проценту должников между категориями по доходу не существенная. Делаем вывод, что категория дохода не влияет на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [214]:
# добавим столбцы с расчетом процентов в сводную таблицу
data_pivot_group_purpose['debt_0_group_purpose'] = data_pivot_group_purpose[0] * 100 / (data_pivot_group_purpose[0] + data_pivot_group_purpose[1]) #столбец с процентом возврата в срок
data_pivot_group_purpose['debt_1_group_purpose'] = data_pivot_group_purpose[1] * 100 / (data_pivot_group_purpose[0] + data_pivot_group_purpose[1]) #столбец с процентом по долгу

In [215]:
display(data_pivot_group_purpose.sort_values(by = 'debt_1_group_purpose')) # вывод сводной таблицы по срокам возврата кредита 
                                                                          # по клиентам с разными целями кредита

debt,0,1,debt_0_group_purpose,debt_1_group_purpose
group_purpose,,,,
на приобретение недвижимости,10029,782,92.766627,7.233373
на свадьбу,2138,186,91.996558,8.003442
на получение образования,3643,370,90.779965,9.220035
на приобретение автомобиля,3903,403,90.640966,9.359034


**Вывод**

По сводной таблице можно отметить, что больший процент тех, кто в срок не погашает задолжность по кредиту среди людей, взявших деньги на приобретение автомобиля. Меньший процент по задолжности среди людей взявших кредит на недвижимость. Так как цели кредитов абсолютно разные и охватывают разные сферы жизни людей, можно сказать, что зависимость между целью кредита и его возвратом в срок вовремя есть. Риски банка повышаются, когда выдается кредит на автомобиль и на образование.

## Шаг 4. Общий вывод

По результатам проведенного исследования был сделан вывод, что семейное положение, уровень дохода не влияют на возврат кредита. Риски не возврата кредита вовремя повышаются, когда кредит выдается заемщикам с детьми или на покупку автомобиля и получение образования.